• Knowledge data: Factual information that the system should know
idx_2bc37909


• Context data: Structured instructions, or semantic blueprints, introduced in Chapter 1

# The semantic blueprint

## output

He then played with the mysterious red ball, his eyes locked in cold defiance on the ancient, looming grandfather clock.

TQDM for adding progress bar  

In [1]:
from tqdm.auto import tqdm
import time

for i in tqdm(range(10)):
    time.sleep(0.2)  # simulating work


/home/ujjwal/projects/.venv/lib/python3.11/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
  0%|          | 0/10 [00:00<?, ?it/s]

100%|██████████| 10/10 [00:02<00:00,  4.97it/s]


## Instalation

In [2]:
import json
import time
from tqdm.auto import tqdm
import tiktoken
from pinecone import Pinecone, ServerlessSpec
# general imports required in the notebooks of this book
import re
import textwrap
from IPython.display import display, Markdown
import copy

In [3]:
from llm_clients import google_langchain_client 


In [4]:
import os 
from llm_clients import openai_client


In [5]:
from sentence_transformers import SentenceTransformer

model = SentenceTransformer(
    "sentence-transformers/all-MiniLM-L6-v2",
    device="cuda"  # or "cpu"
)

embeddings = model.encode(
    ["FastAPI is a Python web framework"],
    normalize_embeddings=True
)

print(len(embeddings[0]))  # 384

384


NAMESPACE_CONTEXT will store blueprints

NAMESPACE_KNOWLEDGE will store the facts

In [6]:
from llm_clients import google_langchain_client 
from llm_clients import pine_cone_client
from llm_clients import cohere_chat_client , cohere_embedding_client 

In [7]:
## indexs 
index_name= 'genai-mas-mcp-01'
NAMESPACE_KNOWLEDGE = "knowledgeStore"
NAMESPACE_CONTEXT= "ContextLibrary"

In [8]:
spec = ServerlessSpec(cloud='aws', region='us-east-1')

In [9]:
cohere_embedding_client.model

'embed-english-light-v3.0'

In [10]:
import time 
DELETE_TIMEOUT_SECONDS = 120 

if index_name not in pine_cone_client.list_indexes().names():
    print(f"Indedx {index_name} not found . creating new serverless index. ...")
    pine_cone_client.create_index( # Sends async index creation request ,Index is not immediately usable
        name=index_name, 
        dimension=384,
        metric='cosine',
        spec= spec
    )
    index= pine_cone_client(index_name)

    # wait  for the index to be ready
    while not pine_cone_client.describe_index(index_name).status['ready']: # Waits until Pinecone backend finishes provisioning
        print("waiting for index to be ready........")
        time.sleep(2)

    print("Index  create sucessfull . it is new and empty")


## If the index already exists, we can choose whether to clear out existing data. For this demo,
## we’ll start fresh each time by deleting the contents of both namespaces:

else : 
    print(f"{index_name} exist , clearing namespce for fresh start")
    index= pine_cone_client.Index(index_name) 
    namespace_to_clear= [NAMESPACE_CONTEXT , NAMESPACE_KNOWLEDGE]

    for namespace in namespace_to_clear:
        stats= index.describe_index_stats()
        namespace_stats= stats.namespaces.get(namespace)


        if namespace in stats.namespaces and namespace_stats.vector_count > 0 :
            print(f"clear namespace {namespace}")
            index.delete(delete_all=True , namespace= namespace)  ## removes all vector inside namespaces
            start_time= time.time() 

            while True: ##  delete is async request this loop   waits untill  it deletes all the vector inside namespace to zero it does't delete , it just waits untill above delete procecss  finish
                stats= index.describe_index_stats()
                namespace_stats= stats.namespaces.get(namespace)

                if not namespace_stats or namespace_stats.vector_count == 0 :
                    print(f"Namespace {namespace} cleared")
                    break 

                if time.time() - start_time > DELETE_TIMEOUT_SECONDS:
                    raise TimeoutError (f" Time out error while clearning namespace {namespace}"
                    f" remaining vector {namespace_stats.vector_count}")

                
        else : 
            print(f"Namespace {namespace} already exist")




# final index 
index= pine_cone_client.Index(index_name)
print("Index is ready to use")

genai-mas-mcp-01 exist , clearing namespce for fresh start
clear namespace ContextLibrary
Namespace ContextLibrary cleared
clear namespace knowledgeStore
Namespace knowledgeStore cleared
Index is ready to use


In [11]:
import json

context_blueprints = [
    {
        "id": "blueprint_suspense_narrative",
        "description": (
            "A precise Semantic Blueprint designed to generate suspenseful and tense "
            "narratives, suitable for children's stories. Focuses on atmosphere, "
            "perceived threats, and emotional impact. Ideal for creative writing."
        ),
        "blueprint": json.dumps(
            {
                "scene_goal": "Increase tension and create suspense.",
                "style_guide": (
                    "Use short, sharp sentences. Focus on sensory details "
                    "(sounds, shadows). Maintain a slightly eerie but "
                    "age-appropriate tone."
                ),
                "participants": [
                    {
                        "role": "Agent",
                        "description": "The protagonist experiencing the events."
                    },
                    {
                        "role": "Source_of_Threat",
                        "description": "The underlying danger or mystery."
                    }
                ],
                "instruction": (
                    "Rewrite the provided facts into a narrative adhering "
                    "strictly to the scene_goal and style_guide."
                )
            },
            ensure_ascii=False
        )
    }
]


In [12]:
knowledge_data_raw = """
Space exploration refers to the systematic effort to understand outer space using astronomy, physics, engineering, and advanced space technologies. It is not limited to human travel; it also includes satellites, probes, telescopes, and robotic missions designed to observe and experiment beyond Earth’s atmosphere. This field exists to answer fundamental questions about the origin of the universe, planetary systems, and the possibility of life beyond Earth.

The modern era of space exploration began in the mid-20th century during the Cold War, when technological rivalry between the Soviet Union and the United States triggered what became known as the Space Race. The Soviet launch of Sputnik 1 in 1957 marked the first artificial satellite to orbit Earth, proving that spaceflight was technically achievable. This event reshaped military strategy, communications, and scientific research worldwide. The race culminated in 1969 when NASA’s Apollo 11 mission successfully landed humans on the Moon, a milestone that demonstrated precision navigation, life-support systems, and large-scale engineering coordination.

The Apollo program was NASA’s most ambitious human spaceflight initiative, designed not only to land astronauts on the Moon but also to develop technologies required for long-duration space missions. On July 20, 1969, Neil Armstrong became the first human to walk on the lunar surface, collecting samples and deploying scientific instruments that significantly expanded understanding of the Moon’s geology and its relationship to Earth.

Beyond human missions, robotic exploration has played a central role in deep-space science. The Juno spacecraft, launched by NASA, orbits Jupiter to study its internal structure, gravitational field, atmosphere, and powerful magnetic environment. Unlike earlier probes, Juno relies on large solar panels rather than nuclear power, demonstrating that solar energy can support missions even at great distances from the Sun.

Mars exploration represents another major focus of modern space science. Robotic rovers act as mobile laboratories, analyzing soil, rocks, and atmospheric conditions. NASA’s Perseverance rover is specifically designed to investigate whether Mars ever supported microbial life, while also testing technologies for future human missions. It carried the Ingenuity helicopter, which achieved the first powered flight on another planet, proving that aerial exploration is possible beyond Earth.

Together, these missions show how space exploration has evolved from political competition into a long-term scientific and technological endeavor aimed at expanding human knowledge and enabling future interplanetary travel.

"""

## Helper function for Chunking and Embedding

In [13]:
from llm_clients import cohere_embedding_client , cohere_chat_client 


In [14]:
## an examples of encode decoded of openai
import tiktoken

tokenizer = tiktoken.get_encoding("cl100k_base")
print(tokenizer.encode("hifadfa"))
print(tokenizer.decode([71, 333, 329, 3716]))

[71, 333, 329, 3716]
hifadfa


In [15]:
def tokenize(text , model="command-a-03-2025"):
    return cohere_chat_client.client.tokenize(text= text, model= model)

In [16]:
text= "hellow my name is ujjwal"
answer = cohere_chat_client.client.tokenize(text=text , model="command-a-03-2025" )
print(answer)

print("**************************************************************************")
def tokenize(text , model="command-a-03-2025"):
    return cohere_chat_client.client.tokenize(text= text, model= model).tokens


def detokenize(tokens, model="command-a-03-2025"):
    return cohere_chat_client.client.detokenize(tokens=tokens, model=model).text



print(tokenize("testfadfad"))
detokenize(tokenize("testfadfad"))

tokens=[14043, 1782, 2970, 2773, 1801, 3532, 66202, 15251] token_strings=[] meta=None
**************************************************************************
[3860, 77, 143972, 1731]


'testfadfad'

## chunk text

In [17]:
def chunk_text(text, chunk_size=400, overlap=50):
    tokens= tokenize(text)
    chunks= [] 
    for i in range(0 , len(tokens), chunk_size - overlap):
        chunk_tokens= tokens[i:i + chunk_size]
        chunk_text= detokenize(chunk_tokens)

        ## clean up 
        chunk_text= chunk_text.replace("\n", " ").strip()
        if chunk_text:
            chunks.append(chunk_text)

    return chunks 

### we neeed to convert chunks to embeddings sometimes apis call can fails to we willt Retry

In [18]:
from llm_clients import  cohere_embedding_client 
vectors = cohere_embedding_client.embed_documents(["fadfadfa fadsfada "]) # embed_documents is list[str]
vectors

[[-0.1038208,
  0.0006337166,
  -0.0019111633,
  -0.0034179688,
  0.04272461,
  0.05517578,
  0.023101807,
  -0.045532227,
  0.033203125,
  0.06414795,
  0.046417236,
  0.032104492,
  0.0066070557,
  0.096191406,
  -0.056274414,
  -0.044708252,
  -0.02003479,
  0.054260254,
  -0.005554199,
  0.050598145,
  0.06945801,
  -0.0143966675,
  0.014427185,
  -0.047607422,
  -0.04336548,
  0.035980225,
  0.021469116,
  0.006881714,
  -0.028289795,
  -0.029022217,
  -0.06695557,
  0.02204895,
  0.068603516,
  0.003129959,
  0.06072998,
  -0.10888672,
  -0.015068054,
  0.047424316,
  -0.027069092,
  0.06774902,
  -0.052612305,
  0.0011930466,
  0.041381836,
  -0.0287323,
  0.024414062,
  0.06591797,
  0.015777588,
  0.033447266,
  0.089782715,
  0.07336426,
  0.03640747,
  -0.009666443,
  -0.02142334,
  0.03262329,
  0.031677246,
  0.0736084,
  -0.004447937,
  -0.024871826,
  0.04623413,
  0.06536865,
  -0.023712158,
  0.02420044,
  -0.107177734,
  0.040649414,
  0.02784729,
  0.13452148,
  -0.0

In [19]:
from trace import CoverageResults
from tenacity import retry, stop_after_attempt, wait_random_exponential

@retry(
    wait= wait_random_exponential(min=1 , max=60),
    stop= stop_after_attempt(6)
    
    )

def get_embedding_batch(texts):
    texts = [t.replace("\n", " ") for t in texts]
    response= cohere_embedding_client.embed_documents(texts)
    return response



# Final data processing


In [21]:
print(f"\nProcessing and uploading Context Library to namespace:{NAMESPACE_CONTEXT}")
vectors_context = []

## for loop with progress bar

for item in tqdm(context_blueprints):
    ## here we will embedeed the description only , siince we are uisnig tqdm , this will show how much embbeddign is completed
    embedding= get_embedding_batch(item['description'][0])
    vectors_context.append({
        "id": item['id'],
        "values": embedding[0],
        "metadata": {
            "description": item['description'],
            "blueprint_json": item['blueprint']
        }
    })




Processing and uploading Context Library to namespace:ContextLibrary


100%|██████████| 1/1 [00:00<00:00,  1.40it/s]


In [22]:
type(vectors_context[0])

dict

In [23]:
## upsert i.e Update if Id already exist, else  create new id 
if vectors_context:
    index.upsert(vectors=vectors_context,namespace=  NAMESPACE_CONTEXT)
    print(f"sucessfully uploade {len(vectors_context)}")



sucessfully uploade 1


##   now  knowlode Base

In [24]:
print(knowledge_data_raw)


Space exploration refers to the systematic effort to understand outer space using astronomy, physics, engineering, and advanced space technologies. It is not limited to human travel; it also includes satellites, probes, telescopes, and robotic missions designed to observe and experiment beyond Earth’s atmosphere. This field exists to answer fundamental questions about the origin of the universe, planetary systems, and the possibility of life beyond Earth.

The modern era of space exploration began in the mid-20th century during the Cold War, when technological rivalry between the Soviet Union and the United States triggered what became known as the Space Race. The Soviet launch of Sputnik 1 in 1957 marked the first artificial satellite to orbit Earth, proving that spaceflight was technically achievable. This event reshaped military strategy, communications, and scientific research worldwide. The race culminated in 1969 when NASA’s Apollo 11 mission successfully landed humans on the Mo

In [25]:
knowledge_chunks = chunk_text(knowledge_data_raw)
len(knowledge_chunks) ## it should be 2

2

In [26]:
print(f"\nProcessing and uploading Knowledge Base to namespace:{NAMESPACE_KNOWLEDGE}")
vectors_knowledge = []
knowledge_chunks = chunk_text(knowledge_data_raw)
batch_size = 100
for i in tqdm(range(0, len(knowledge_chunks), batch_size)):
    batch_texts = knowledge_chunks[i:i+batch_size]
    batch_embeddings = get_embedding_batch(batch_texts)

    batch_vectors = []
    for j, embedding in enumerate(batch_embeddings):
        chunk_id = f"knowledge_chunk_{i+j}"
        batch_vectors.append({
                                "id": chunk_id,
                                "values": embedding,
                                "metadata": {
                                        "text": batch_texts[j]

                                }
        })

    
    # Upsert the batch
    index.upsert(vectors=batch_vectors, namespace=NAMESPACE_KNOWLEDGE)


print(f"Successfully uploaded {len(knowledge_chunks)} knowledge vectors.")
    



Processing and uploading Knowledge Base to namespace:knowledgeStore


100%|██████████| 1/1 [00:01<00:00,  1.14s/it]

Successfully uploaded 2 knowledge vectors.


### Rag works is complete Now we will use this Rag to build Context Aware System